This is a companion notebook for the book [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff). For readability, it only contains runnable code blocks and section titles, and omits everything else in the book: text paragraphs, figures, and pseudocode.

**If you want to be able to follow what's going on, I recommend reading the notebook side by side with your copy of the book.**

This notebook was generated for TensorFlow 2.6.

## Modern convnet architecture patterns

### Modularity, hierarchy, and reuse

### Residual connections

**Residual block where the number of filters changes**

In [ ]:
from tensorflow import keras # importing keras from tensorflow 
from tensorflow.keras import layers # importing layers from tensorflow.keras

inputs = keras.Input(shape=(32, 32, 3)) # specifying the input shape of the model as 32x32x3 because we are using CIFAR-10 dataset which has 32x32x3 images 
x = layers.Conv2D(32, 3, activation="relu")(inputs) # adding a convolutional layer with 32 filters of size 3x3 and relu activation function
residual = x # saving the output of the convolutional layer in a variable called residual
x = layers.Conv2D(64, 3, activation="relu", padding="same")(x) # adding another convolutional layer with 64 filters of size 3x3 and relu activation function
residual = layers.Conv2D(64, 1)(residual) # adding a convolutional layer with 64 filters of size 1x1 to the residual variable
x = layers.add([x, residual]) # adding the output of the second convolutional layer and the residual variable

**Case where target block includes a max pooling layer**

In [ ]:
inputs = keras.Input(shape=(32, 32, 3)) # specifying the input shape of the model as 32x32x3 because we are using CIFAR-10 dataset which has 32x32x3 images
x = layers.Conv2D(32, 3, activation="relu")(inputs) # adding a convolutional layer with 32 filters of size 3x3 and relu activation function
residual = x # saving the output of the convolutional layer in a variable called residual
x = layers.Conv2D(64, 3, activation="relu", padding="same")(x) # adding another convolutional layer with 64 filters of size 3x3 and relu activation function
x = layers.MaxPooling2D(2, padding="same")(x) # adding a max pooling layer with a pool size of 2x2
residual = layers.Conv2D(64, 1, strides=2)(residual) # adding a convolutional layer with 64 filters of size 1x1 and a stride of 2 to the residual variable
x = layers.add([x, residual]) # adding the output of the max pooling layer and the residual variable

In [ ]:
inputs = keras.Input(shape=(32, 32, 3)) # specifying the input shape of the model as 32x32x3 because we are using CIFAR-10 dataset which has 32x32x3 images
x = layers.Rescaling(1./255)(inputs) # rescaling the input data to be between 0 and 1

def residual_block(x, filters, pooling=False): # defining a function called residual_block that takes in the input x, the number of filters and a pooling parameter
    residual = x # saving the input in a variable called residual
    x = layers.Conv2D(filters, 3, activation="relu", padding="same")(x) # adding a convolutional layer with the specified number of filters, a filter size of 3x3, relu activation function and same padding
    x = layers.Conv2D(filters, 3, activation="relu", padding="same")(x) # adding another convolutional layer with the specified number of filters, a filter size of 3x3, relu activation function and same padding
    if pooling: # if the pooling parameter is True
        x = layers.MaxPooling2D(2, padding="same")(x) # add a max pooling layer with a pool size of 2x2
        residual = layers.Conv2D(filters, 1, strides=2)(residual) # add a convolutional layer with the specified number of filters, a filter size of 1x1 and a stride of 2 to the residual variable
    elif filters != residual.shape[-1]: # if the number of filters is not equal to the number of channels in the residual variable
        residual = layers.Conv2D(filters, 1)(residual) # add a convolutional layer with the specified number of filters and a filter size of 1x1 to the residual variable
    x = layers.add([x, residual]) # add the output of the second convolutional layer and the residual variable
    return x # returning the output

x = residual_block(x, filters=32, pooling=True) # calling the residual_block function with the input x, 32 filters and pooling set to True
x = residual_block(x, filters=64, pooling=True) # calling the residual_block function with the input x, 64 filters and pooling set to True
x = residual_block(x, filters=128, pooling=False) # calling the residual_block function with the input x, 128 filters and pooling set to False

x = layers.GlobalAveragePooling2D()(x) # adding a global average pooling layer
outputs = layers.Dense(1, activation="sigmoid")(x) # adding a dense layer with 1 unit and a sigmoid activation function
model = keras.Model(inputs=inputs, outputs=outputs) # creating a model with the specified inputs and outputs
model.summary() # printing a summary of the model

### Batch normalization

### Depthwise separable convolutions

### Putting it together: A mini Xception-like model

In [ ]:
from google.colab import files # importing files from google.colab
files.upload() # uploading the kaggle.json file

In [ ]:
!mkdir ~/.kaggle # creating a directory called .kaggle
!cp kaggle.json ~/.kaggle/ # copying the kaggle.json file to the .kaggle directory
!chmod 600 ~/.kaggle/kaggle.json # changing the permissions of the kaggle.json file
!kaggle competitions download -c dogs-vs-cats # downloading the dogs-vs-cats competition data
!unzip -qq train.zip # unzipping the train.zip file

In [ ]:
import os, shutil, pathlib # importing os, shutil and pathlib
from tensorflow.keras.utils import image_dataset_from_directory # importing image_dataset_from_directory from tensorflow.keras.utils

original_dir = pathlib.Path("train") # specifying the original directory where the data is stored
new_base_dir = pathlib.Path("cats_vs_dogs_small") # specifying the new base directory where the data will be stored

def make_subset(subset_name, start_index, end_index): # defining a function called make_subset that takes in the subset name, the start index and the end index
    for category in ("cat", "dog"): # looping through the categories
        dir = new_base_dir / subset_name / category # specifying the directory where the subset data will be stored
        os.makedirs(dir) # creating the directory
        fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)] # creating a list of file names
        for fname in fnames: # looping through the file names
            shutil.copyfile(src=original_dir / fname, # copying the files from the original directory to the new directory
                            dst=dir / fname) # specifying the source and destination paths

make_subset("train", start_index=0, end_index=1000) # calling the make_subset function to create the training subset
make_subset("validation", start_index=1000, end_index=1500) # calling the make_subset function to create the validation subset
make_subset("test", start_index=1500, end_index=2500) # calling the make_subset function to create the test subset

train_dataset = image_dataset_from_directory( # creating a training dataset using the image_dataset_from_directory function
    new_base_dir / "train", # specifying the directory where the training data is stored
    image_size=(180, 180), # specifying the image size
    batch_size=32) # specifying the batch size
validation_dataset = image_dataset_from_directory( # creating a validation dataset using the image_dataset_from_directory function
    new_base_dir / "validation", # specifying the directory where the validation data is stored
    image_size=(180, 180), # specifying the image size
    batch_size=32) # specifying the batch size
test_dataset = image_dataset_from_directory( # creating a test dataset using the image_dataset_from_directory function
    new_base_dir / "test", # specifying the directory where the test data is stored
    image_size=(180, 180), # specifying the image size
    batch_size=32) # specifying the batch size

In [ ]:
data_augmentation = keras.Sequential( # creating a data augmentation layer
    [
        layers.RandomFlip("horizontal"), # adding a random flip layer
        layers.RandomRotation(0.1), # adding a random rotation layer
        layers.RandomZoom(0.2), # adding a random zoom layer
    ]
)

In [ ]:
inputs = keras.Input(shape=(180, 180, 3)) # specifying the input shape of the model as 180x180x3
x = data_augmentation(inputs) # adding the data augmentation layer

x = layers.Rescaling(1./255)(x) # rescaling the input data to be between 0 and 1
x = layers.Conv2D(filters=32, kernel_size=5, use_bias=False)(x) # adding a convolutional layer with 32 filters of size 5x5 and no bias

for size in [32, 64, 128, 256, 512]: # looping through the sizes
    residual = x # saving the input in a variable called residual

    x = layers.BatchNormalization()(x) # adding a batch normalization layer
    x = layers.Activation("relu")(x) # adding a relu activation layer
    x = layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(x) # adding a separable convolutional layer with the specified number of filters, a filter size of 3x3, same padding and no bias

    x = layers.BatchNormalization()(x) # adding a batch normalization layer
    x = layers.Activation("relu")(x) # adding a relu activation layer
    x = layers.SeparableConv2D(size, 3, padding="same", use_bias=False)(x) # adding another separable convolutional layer with the specified number of filters, a filter size of 3x3, same padding and no bias

    x = layers.MaxPooling2D(3, strides=2, padding="same")(x) # adding a max pooling layer with a pool size of 3x3, a stride of 2 and same padding

    residual = layers.Conv2D( # adding a convolutional layer to the residual variable
        size, 1, strides=2, padding="same", use_bias=False)(residual) # with the specified number of filters, a filter size of 1x1, a stride of 2, same padding and no bias
    x = layers.add([x, residual]) # adding the output of the max pooling layer and the residual variable

x = layers.GlobalAveragePooling2D()(x) # adding a global average pooling layer
x = layers.Dropout(0.5)(x) # adding a dropout layer with a rate of 0.5
outputs = layers.Dense(1, activation="sigmoid")(x) # adding a dense layer with 1 unit and a sigmoid activation function
model = keras.Model(inputs=inputs, outputs=outputs) # creating a model with the specified inputs and outputs

In [ ]:
model.compile(loss="binary_crossentropy", # compiling the model with binary crossentropy loss
              optimizer="rmsprop", # using the rmsprop optimizer
              metrics=["accuracy"]) # using accuracy as the metric
history = model.fit( # training the model
    train_dataset, # using the training dataset
    epochs=100, # training for 100 epochs
    validation_data=validation_dataset) # using the validation dataset